In [1]:
#imports
#imports 
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup
import lxml
import html5lib
import string
import re
import torch
from torch import nn
import time

In [47]:
#create dictionary of league with level key
league_level = {'INT':'AAA','PCL':'AAA','EAS':'AA','SOU':'AA','TXS':'AA','CLF':'A+','CRL':'A+','FSL':'A+','MDW':'A','SAL':'A','NYP':'A-',
                'NWN':'A-','APL':'Rk_y','PIO':'Rk_y'}
for k in league_level.keys():
    print(k)

INT
PCL
EAS
SOU
TXS
CLF
CRL
FSL
MDW
SAL
NYP
NWN
APL
PIO


In [ ]:
year_list = list(range(2000,2020,1))

results_list = []
for k in league_level.keys():
    for year in year_list:
        url = 'http://claydavenport.com/stats/webpages/'+str(year)+'/'+str(year)+'page'+str(k)+'realALL.shtml'
        print(url)
        tables = pd.read_html(url)
        print(len(tables[1].index))
        results_list.append(tables[1])
        time.sleep(1)
    
league_frame_hitting = pd.concat(results_list, ignore_index=True)

http://claydavenport.com/stats/webpages/2000/2000pageINTrealALL.shtml
348
http://claydavenport.com/stats/webpages/2001/2001pageINTrealALL.shtml
365
http://claydavenport.com/stats/webpages/2002/2002pageINTrealALL.shtml
429
http://claydavenport.com/stats/webpages/2003/2003pageINTrealALL.shtml
449
http://claydavenport.com/stats/webpages/2004/2004pageINTrealALL.shtml
426
http://claydavenport.com/stats/webpages/2005/2005pageINTrealALL.shtml
433
http://claydavenport.com/stats/webpages/2006/2006pageINTrealALL.shtml
439
http://claydavenport.com/stats/webpages/2007/2007pageINTrealALL.shtml
454
http://claydavenport.com/stats/webpages/2008/2008pageINTrealALL.shtml
472
http://claydavenport.com/stats/webpages/2009/2009pageINTrealALL.shtml
522
http://claydavenport.com/stats/webpages/2010/2010pageINTrealALL.shtml
499
http://claydavenport.com/stats/webpages/2011/2011pageINTrealALL.shtml
477
http://claydavenport.com/stats/webpages/2012/2012pageINTrealALL.shtml
512
http://claydavenport.com/stats/webpage

In [46]:
#print(league_frame_hitting.head())
#print(league_frame_hitting.columns)

league_frame_hitting.columns = list(league_frame_hitting.columns[:28])+['DRS1', 'Defense2','DRS2'] #rename columns

#add a column that reports the level the player played at
league_frame_hitting['Lev'] = [league_level[k] for k in league_frame_hitting['Lge']]
#league_frame_pitching['Lev'] = [league_level[k] for k in league_frame_pitching['Lge']]

league_frame_hitting['GP1'] = [float(re.findall(r'\d+', v)[0]) for v in list(league_frame_hitting['Defense'])]

league_frame_hitting['Position1'] = [re.findall(r'-([1-9A-Z]+)', v)[0] for v in list(league_frame_hitting['Defense'])]

#create player age and clean name, will use for future matching against fangraphs or bref ids
league_frame_hitting['Name'] = [x.strip().replace('_', ' ') for x in league_frame_hitting['Name']]
league_frame_hitting['Name'] = league_frame_hitting['Name'].str.replace(r'[^A-Za-z\s]+', '') #replace apostrophes and periods in player names
#print(league_frame_hitting)
#attempt to group player by year-level
league_frame_hitting['PA'] = league_frame_hitting['AB']+league_frame_hitting['BB'] 
#first split levels into columns with booleans
frame_to_add = pd.get_dummies(league_frame_hitting['Lev']).mul(league_frame_hitting['PA'], axis=0)#*t())
league_frame_hitting = pd.concat([league_frame_hitting,frame_to_add], axis=1)
#league_frame_hitting = league_frame_hitting.iloc[:,6:]
#print(league_frame_hitting) 
league_frame_hitting.to_csv('davcheck.csv')
#multiply each column by games played

#group sum

league_frame_hitting_group = league_frame_hitting.groupby(['Name','Age','Year'],as_index=False).sum()

#dived rows by rowwise sum pandas
#league_frame_hitting_group["sum"] = league_frame_hitting_group[['A','A+','A-','AA','AAA','Rk_y']].sum(axis=1)
league_frame_hitting_group.loc[:,"A":"Rk_y"] = league_frame_hitting_group.loc[:,"A":"Rk_y"].div(league_frame_hitting_group["PA"], axis=0)
print(league_frame_hitting_group)
league_frame_hitting_group.to_csv('davenporthitting.csv')
#convert to weights across grouped columns
#league_frame_hitting_grouped['rowsum'] = 
#export to csv

                Name  Age  Year   AB    H  DB  TP  HR  BB   SO  ...  DRS1  \
0      A.J. Battisto   25  2009    1    0   0   0   0   0    1  ...     0   
1      A.J. Battisto   29  2013    3    0   0   0   0   0    3  ...     0   
2       A.J. Burnett   27  2004    2    0   0   0   0   0    1  ...     0   
3          A.J. Cole   21  2013    5    0   0   0   0   0    3  ...     0   
4          A.J. Cole   22  2014   15    3   0   0   0   0    3  ...    -1   
...              ...  ...   ...  ...  ...  ..  ..  ..  ..  ...  ...   ...   
62168  Zoilo Almonte   21  2010  465  125  23   5  13  44  130  ...    -3   
62169  Zoilo Almonte   22  2011  434  120  26   4  15  45  105  ...    -4   
62170  Zoilo Almonte   23  2012  419  116  23   1  21  25  103  ...    10   
62171  Zoilo Almonte   24  2013  259   77  12   1   6  30   47  ...    -3   
62172  Zoilo Almonte   25  2014  421  110  18   1  18  29  105  ...    -4   

       DRS2   GP1   PA        A        A+   A-        AA       AAA  Rk_y  


In [5]:
'''
results_list = []
for k in league_level.keys():
    for year in year_list:
        url = 'http://claydavenport.com/stats/webpages/'+str(year)+'/'+str(year)+'ppage'+str(k)+'realALL.shtml'
        print(url)
        tables = pd.read_html(url)
        print(len(tables[1].index))
        results_list.append(tables[1])
        time.sleep(1)
league_frame_pitching = pd.concat(results_list, ignore_index=True)
'''

http://claydavenport.com/stats/webpages/2000/2000ppageINTrealALL.shtml
384
http://claydavenport.com/stats/webpages/2001/2001ppageINTrealALL.shtml
387
http://claydavenport.com/stats/webpages/2002/2002ppageINTrealALL.shtml
395
http://claydavenport.com/stats/webpages/2003/2003ppageINTrealALL.shtml
375
http://claydavenport.com/stats/webpages/2004/2004ppageINTrealALL.shtml
400
http://claydavenport.com/stats/webpages/2005/2005ppageINTrealALL.shtml
378
http://claydavenport.com/stats/webpages/2006/2006ppageINTrealALL.shtml
398
http://claydavenport.com/stats/webpages/2007/2007ppageINTrealALL.shtml
422
http://claydavenport.com/stats/webpages/2008/2008ppageINTrealALL.shtml
404
http://claydavenport.com/stats/webpages/2009/2009ppageINTrealALL.shtml
449
http://claydavenport.com/stats/webpages/2010/2010ppageINTrealALL.shtml
408
http://claydavenport.com/stats/webpages/2011/2011ppageINTrealALL.shtml
428
http://claydavenport.com/stats/webpages/2012/2012ppageINTrealALL.shtml
435
http://claydavenport.com/

279
http://claydavenport.com/stats/webpages/2010/2010ppageCLFrealALL.shtml
267
http://claydavenport.com/stats/webpages/2011/2011ppageCLFrealALL.shtml
273
http://claydavenport.com/stats/webpages/2012/2012ppageCLFrealALL.shtml
285
http://claydavenport.com/stats/webpages/2013/2013ppageCLFrealALL.shtml
289
http://claydavenport.com/stats/webpages/2014/2014ppageCLFrealALL.shtml
289
http://claydavenport.com/stats/webpages/2015/2015ppageCLFrealALL.shtml
315
http://claydavenport.com/stats/webpages/2016/2016ppageCLFrealALL.shtml
346
http://claydavenport.com/stats/webpages/2017/2017ppageCLFrealALL.shtml
278
http://claydavenport.com/stats/webpages/2018/2018ppageCLFrealALL.shtml
310
http://claydavenport.com/stats/webpages/2019/2019ppageCLFrealALL.shtml
307
http://claydavenport.com/stats/webpages/2000/2000ppageFSLrealALL.shtml
328
http://claydavenport.com/stats/webpages/2001/2001ppageFSLrealALL.shtml
325
http://claydavenport.com/stats/webpages/2002/2002ppageFSLrealALL.shtml
329
http://claydavenport.

256
http://claydavenport.com/stats/webpages/2000/2000ppageAPLrealALL.shtml
162
http://claydavenport.com/stats/webpages/2001/2001ppageAPLrealALL.shtml
181
http://claydavenport.com/stats/webpages/2002/2002ppageAPLrealALL.shtml
182
http://claydavenport.com/stats/webpages/2003/2003ppageAPLrealALL.shtml
186
http://claydavenport.com/stats/webpages/2004/2004ppageAPLrealALL.shtml
210
http://claydavenport.com/stats/webpages/2005/2005ppageAPLrealALL.shtml
185
http://claydavenport.com/stats/webpages/2006/2006ppageAPLrealALL.shtml
193
http://claydavenport.com/stats/webpages/2007/2007ppageAPLrealALL.shtml
173
http://claydavenport.com/stats/webpages/2008/2008ppageAPLrealALL.shtml
204
http://claydavenport.com/stats/webpages/2009/2009ppageAPLrealALL.shtml
197
http://claydavenport.com/stats/webpages/2010/2010ppageAPLrealALL.shtml
208
http://claydavenport.com/stats/webpages/2011/2011ppageAPLrealALL.shtml
211
http://claydavenport.com/stats/webpages/2012/2012ppageAPLrealALL.shtml
225
http://claydavenport.